In [1]:
import os
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import IsolationForest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.spatial import cKDTree
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
pd.set_option('display.max_columns', None)  # Toutes les colonnes
pd.set_option('display.width', None)        # Pas de limite de largeur pour les lignes
pd.set_option('display.max_colwidth', None) # Pas de limite pour la largeur des cellules

def change_column_types(df):
    """
    Change les types de plusieurs colonnes dans un DataFrame pandas.

    Parameters:
        df (pd.DataFrame): Le DataFrame pandas.

    Returns:
        pd.DataFrame: Un nouveau DataFrame avec les colonnes modifiées.
    """
    # Dictionnaire des colonnes et leurs types cibles
    dict_type = {
        'id_mutation': 'string',
        'date_mutation': 'datetime64[ns]',
        'numero_disposition': 'string',
        'nature_mutation': 'string',
        'valeur_fonciere': 'float64',
        'adresse_numero': 'string',
        'adresse_suffixe': 'string',
        'adresse_nom_voie': 'string',
        'adresse_code_voie': 'string',
        'code_postal': 'string',
        'code_commune': 'string',
        'nom_commune': 'string',
        'code_departement': 'string',
        'ancien_code_commune': 'string',
        'ancien_nom_commune': 'string',
        'id_parcelle': 'string',
        'ancien_id_parcelle': 'string',
        'numero_volume': 'string',
        'lot1_numero': 'string',
        'lot1_surface_carrez': 'float64',
        'lot2_numero': 'string',
        'lot2_surface_carrez': 'float64',
        'lot3_numero': 'string',
        'lot3_surface_carrez': 'float64',
        'lot4_numero': 'string',
        'lot4_surface_carrez': 'float64',
        'lot5_numero': 'string',
        'lot5_surface_carrez': 'float64',
        'nombre_lots': 'float64',
        'code_type_local': 'string',
        'type_local': 'string',
        'surface_reelle_bati': 'float64',
        'nombre_pieces_principales': 'float64',
        'code_nature_culture': 'string',
        'nature_culture': 'string',
        'code_nature_culture_speciale': 'string',
        'nature_culture_speciale': 'string',
        'surface_terrain': 'float64',
        'longitude': 'float64',
        'latitude': 'float64'
    }

    # Conversion des types de colonnes
    for column_name, column_type in dict_type.items():
        if column_name in df.columns:  # Vérifie que la colonne existe
            try:
                df[column_name] = df[column_name].astype(column_type)
            except Exception as e:
                print(f"Erreur lors de la conversion de la colonne {column_name}: {e}")

    return df

def data_loader(path, departements=[], annees=[], fraction = None, chunksize=None):
    """
    Charge les fichiers CSV d'un répertoire et les concatène dans un DataFrame pandas.

    Parameters:
        path (str): Chemin vers le dossier contenant les données.
        departements (list): Liste des départements à inclure (ex: [1, 75]).
        annees (list): Liste des années à inclure (ex: [2020, 2021]).
        fraction (float): Fraction des données à charger (ex: 0.1 pour 10%). Si None, charge toutes les données.
        chunksize (int): Taille des chunks à charger (en nombre de lignes). Si None, charge tout le fichier d'un coup.

    Returns:
        pd.DataFrame: Un DataFrame pandas contenant toutes les données chargées.
    """
    df = pd.DataFrame()

    # Liste des années
    annees_list = os.listdir(path) if not annees else [str(annee) for annee in annees]

    for annee in annees_list:
        cur_year = os.path.join(path, annee)

        # Liste des départements
        if not departements:
            departements_list = os.listdir(cur_year)
        else:
            departements_list = [f"{departement:02}.csv.gz" for departement in departements]

        for departement in departements_list:

            file = os.path.join(cur_year, departement)

            try:
                # Charger les données par chunks si chunksize est spécifié
                if chunksize:
                    chunk_list = []
                    for chunk in pd.read_csv(file, chunksize=chunksize, low_memory=False):
                        # Appliquer la fraction sur chaque chunk si nécessaire
                        if fraction is not None:
                            chunk = chunk.sample(frac=fraction, random_state=42)
                        chunk = change_column_types(chunk)
                        chunk_list.append(chunk)

                    temp_df = pd.concat(chunk_list, ignore_index=True)

                else:
                    # Charger le fichier complet si chunksize n'est pas spécifié
                    temp_df = pd.read_csv(file, low_memory=False)
                    if fraction is not None:
                        temp_df = temp_df.sample(frac=fraction, random_state=42)
                    temp_df = change_column_types(temp_df)

                # Concaténer les données
                df = pd.concat([df, temp_df], ignore_index=True)

            except Exception as e:
                print(f"Erreur lors du chargement du fichier {file}: {e}")

    return df.dropna(subset=['valeur_fonciere'])


In [2]:
# Étape 1 : Nettoyage des données (Colonnes nécessaires et NaN)
class DataCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X[[
                'id_mutation','date_mutation', 'type_local', 'surface_reelle_bati',
                'nombre_lots', 'lot1_surface_carrez', 'lot2_surface_carrez',
                'lot3_surface_carrez', 'lot4_surface_carrez', 'lot5_surface_carrez',
                'nombre_pieces_principales', 'surface_terrain', 'longitude', 'latitude',
                'valeur_fonciere'
            ]]
        X = X[
            (X["valeur_fonciere"].notna())
            & (X["longitude"].notna())
            & (X["latitude"].notna())
            & (X["surface_reelle_bati"].notna() | X["surface_terrain"].notna())
            & (X["nombre_lots"] <= 5)

        ]
        return X

# Étape 2 : Création de colonnes calculées
class FeatureCreator(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Ajout de transformations basées sur X
        X = X.assign(
            sin_month=np.sin(2 * np.pi * pd.to_datetime(X["date_mutation"]).dt.month / 12),
            cos_month=np.cos(2 * np.pi * pd.to_datetime(X["date_mutation"]).dt.month / 12),
            year=pd.to_datetime(X["date_mutation"]).dt.year,
            lot1_surface_carrez=X['lot1_surface_carrez'].fillna(0),
            lot2_surface_carrez=X['lot2_surface_carrez'].fillna(0),
            lot3_surface_carrez=X['lot3_surface_carrez'].fillna(0),
            lot4_surface_carrez=X['lot4_surface_carrez'].fillna(0),
            lot5_surface_carrez=X['lot5_surface_carrez'].fillna(0),
            surface_reelle_bati=X['surface_reelle_bati'].fillna(0),
            surface_terrain=X['surface_terrain'].fillna(0)
        )
        X = X.drop('date_mutation',axis = 1)
        X["total_surface_carrez"] = (
            X["lot1_surface_carrez"] +
            X["lot2_surface_carrez"] +
            X["lot3_surface_carrez"] +
            X["lot4_surface_carrez"] +
            X["lot5_surface_carrez"]
        )
        X["total_surface_carrez"] = np.where(
            X["total_surface_carrez"] == 0,
            X["surface_reelle_bati"],
            X["total_surface_carrez"]
        )
        X=X.groupby(by=['id_mutation', 'valeur_fonciere'], as_index=False)\
            .agg({
                'surface_reelle_bati': 'sum',
                'year' : 'first',
                'sin_month' : 'first',
                'cos_month' : 'first',
                'type_local' : lambda x: ', '.join(sorted(x.dropna().unique())) if x.notna().any() else 'None',
                'nombre_lots' : 'max',
                'lot1_surface_carrez': 'mean',
                'lot2_surface_carrez': 'mean',
                'lot3_surface_carrez': 'mean',
                'lot4_surface_carrez': 'mean',
                'lot5_surface_carrez': 'mean',
                'nombre_pieces_principales': 'sum',
                'surface_terrain': 'sum',
                'longitude' : 'first',
                'latitude' : 'first'
            })
        # X["valeur_fonciere_m2"] = np.where(
        #     X["surface_reelle_bati"] != 0,
        #     (X["valeur_fonciere"] / X["surface_reelle_bati"]).round(0),
        #     (X["valeur_fonciere"] / X["surface_terrain"]).round(0)
        # )
        # X["valeur_fonciere_m2_log"] = X["valeur_fonciere_m2"].apply(lambda x: np.log10(x) if x > 0 else np.nan)

        return X.drop('id_mutation',axis = 1)

# Étape 3 : Filtrage des anomalies
class AnomalyFilter(BaseEstimator, TransformerMixin):
    def __init__(self, contamination=0.1):
        self.contamination = contamination
        self.model = IsolationForest(contamination=self.contamination, random_state=42)
        self.anomaly_columns =['surface_reelle_bati', 'nombre_lots', 'surface_terrain', 'nombre_pieces_principales']

    def fit(self, X, y=None):
        self.model.fit(X[self.anomaly_columns])
        return self

    def transform(self, X):
        X["anomalie"] = self.model.predict(X[self.anomaly_columns])
        X = X[X["anomalie"] == 1]
        return X.drop(columns=["anomalie"])

# Étape 4 : ajout des données de densité et de pois
class WeightedPOICountsTransformer(BaseEstimator, TransformerMixin):
    """
    Calcule une moyenne pondérée des colonnes POIs des voisins les plus proches pour chaque point
    en utilisant un arbre k-d (cKDTree).
    """
    def __init__(self, n_neighbors=4, df_grid=None):
        """
        Initialise le transformateur.

        Parameters:
        - poi_columns (list): Liste des noms des colonnes POIs à inclure dans les calculs.
        - n_neighbors (int): Nombre de voisins à prendre en compte pour le calcul des POIs pondérés.
        """
        self.poi_columns = ['densite', 'transport_pois', 'education_pois', 'health_pois', 'food_pois',
                                'shopping_pois', 'park_pois', 'entertainment_pois', 'cultural_pois']

        self.n_neighbors = n_neighbors
        self.df_grid = df_grid

    def fit(self, X, y=None):

        return self

    def transform(self, X, y=None):
        """
        Applique la transformation pour calculer les POIs pondérés sur les données.

        Parameters:
        - X (pandas.DataFrame): DataFrame contenant les coordonnées des points pour lesquels
          les POIs pondérés doivent être calculés.

        Returns:
        - pandas.DataFrame: DataFrame enrichi avec les colonnes pondérées des POIs.
        """
        if self.df_grid is None:
            raise ValueError("df_grid doit être fourni dans fit avant d'appeler transform.")

        # Extraire les coordonnées des deux ensembles
        latitudes_data = X['latitude'].values
        longitudes_data = X['longitude'].values

        latitudes_grid = self.df_grid['lat'].values
        longitudes_grid = self.df_grid['lon'].values

        assert np.all(np.isfinite(longitudes_grid)), "longitudes_grid contient des valeurs non finies."
        assert np.all(np.isfinite(latitudes_grid)), "latitudes_grid contient des valeurs non finies."
        assert np.all(np.isfinite(longitudes_data)), "longitudes_data contient des valeurs non finies."
        assert np.all(np.isfinite(latitudes_data)), "latitudes_data contient des valeurs non finies."

        # Créer un cKDTree pour une recherche rapide sur df_grid
        tree = cKDTree(np.vstack((longitudes_grid, latitudes_grid)).T)

        # Chercher les n_neighbors voisins les plus proches pour chaque point de X
        distances, indices = tree.query(np.vstack((longitudes_data, latitudes_data)).T, k=self.n_neighbors)

        # Calculer les poids en fonction de l'inverse des distances
        weights = 1 / np.where(distances == 0, 1e-10, distances)  # Évite la division par zéro
        normalized_weights = weights / weights.sum(axis=1, keepdims=True)

        # Calculer les moyennes pondérées pour chaque colonne POI
        for col in self.poi_columns:
            poi_values = self.df_grid[col].values  # Utiliser les POIs de df_grid
            # Récupérer les valeurs des voisins pour cette colonne
            neighbors_poi = poi_values[indices]
            # Calculer la moyenne pondérée
            X[f"{col}_weighted"] = np.floor((neighbors_poi * normalized_weights).sum(axis=1))

        return X


# Pipeline complète
pipeline_preprocess = Pipeline(steps=[
    ("cleaner", DataCleaner()),
    ("feature_creator", FeatureCreator()),
    ("anomaly_filter", AnomalyFilter(contamination=0.1)),
    ('weighted_poi', WeightedPOICountsTransformer(n_neighbors=4)),
])

# pipeline_preprocess_test = Pipeline(steps=[
#     ("cleaner", DataCleaner()),
#     ("feature_creator", FeatureCreator()),
#     ("anomaly_filter", AnomalyFilter(contamination=0.1)),
#     ("quantile_filter", QuantileFilter()),
#     ("statistical_filter", StatisticalFilter(sigma_multiplier=3)),
#     ('weighted_poi', WeightedPOICountsTransformer(n_neighbors=4)),
#     ('feature_remover', FeatureRemover())
# ])

In [3]:
path = 'data_dvf'
df = data_loader(path,annees=['2023'],departements=['75']) #

df_grid = pd.read_csv('data_pop_density/dataframe_densite&amenities_radius=500.csv')
df_grid.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
pipeline_preprocess.set_params(weighted_poi__df_grid = df_grid)

df = pipeline_preprocess.fit_transform(df)
df

In [ ]:
df.shape

In [6]:
# Appliquer la pipeline
X = df.drop('valeur_fonciere',axis = 1)
y= df['valeur_fonciere']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
X_test.shape , y_test.shape

In [ ]:
X_train.shape, y_train.shape

In [9]:
# Encoding and imputer Pipeline

categorical_columns_onehot = ['type_local'] # Columns that need OneHotEncoding
numerical_columns = ['surface_reelle_bati', 'year', 'sin_month',
       'cos_month', 'nombre_lots', 'lot1_surface_carrez',
       'lot2_surface_carrez', 'lot3_surface_carrez', 'lot4_surface_carrez',
       'lot5_surface_carrez', 'nombre_pieces_principales', 'surface_terrain',
       'longitude', 'latitude', 'densite_weighted', 'transport_pois_weighted',
       'education_pois_weighted', 'health_pois_weighted', 'food_pois_weighted',
       'shopping_pois_weighted', 'park_pois_weighted',
       'entertainment_pois_weighted', 'cultural_pois_weighted']
unique_categories = [X[col].dropna().unique() for col in categorical_columns_onehot]
onehot_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', categories=unique_categories))
])

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=pd.NA,strategy='most_frequent')),  # Remplit les NaN avec 0
    ('scaler', MinMaxScaler())                 # Standardisation
])

# Encoding pipeline

column_transformer =  ColumnTransformer(
    transformers=[
        ('onehot', onehot_pipeline, categorical_columns_onehot),
        ('numeric', numeric_pipeline, numerical_columns)
    ]
)

def build_xgboost_model():
    pipeline = Pipeline(steps=[
        ('encoding', column_transformer),
        ('model', XGBRegressor(
            objective='reg:squarederror',
            n_estimators=500,
            learning_rate=0.01,
            max_depth=10,
            subsample = 0.8
        ))
    ])
    return pipeline


In [ ]:
xgb_model = build_xgboost_model()
xgb_model

In [ ]:
xgb_model.fit(X_train, y_train)

In [ ]:
import seaborn as sns
y_pred = xgb_model.predict(X_train)
print("Mean Squared Error (MSE):", mean_squared_error(y_train, y_pred))
print("R2 Score:", r2_score(y_train, y_pred))
sns.scatterplot(x=y_train,y=y_pred)

In [ ]:
import seaborn as sns
y_pred = xgb_model.predict(X_test)
print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
print("R2 Score:", r2_score(y_test, y_pred))
sns.scatterplot(x=y_test,y=y_pred)

In [16]:
def plot_train_test_predictions(y_train, y_test, X_train, X_test, model, save_path):
    """
    Génère une figure avec deux graphiques côte à côte : prédictions pour l'ensemble d'entraînement et de test.

    Parameters:
    - y_train : array-like, les vraies valeurs de l'ensemble d'entraînement.
    - y_test : array-like, les vraies valeurs de l'ensemble de test.
    - X_train : array-like, les features de l'ensemble d'entraînement.
    - X_test : array-like, les features de l'ensemble de test.
    - model : object, le modèle entraîné qui possède une méthode .predict().
    - save_path : str, chemin pour sauvegarder la figure générée.

    Returns:
    - None. La figure est sauvegardée à l'emplacement spécifié par save_path.
    """
    # Prédictions pour les ensembles d'entraînement et de test
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    # Calcul des métriques pour l'ensemble d'entraînement
    mse_train = mean_squared_error(y_train, y_pred_train)
    r2_train = r2_score(y_train, y_pred_train)

    # Calcul des métriques pour l'ensemble de test
    mse_test = mean_squared_error(y_test, y_pred_test)
    r2_test = r2_score(y_test, y_pred_test)

    # Création de la figure
    plt.figure(figsize=(14, 6))

    # Graphique 1 : Ensemble d'entraînement
    plt.subplot(1, 2, 1)
    sns.scatterplot(x=y_train, y=y_pred_train, alpha=0.6, edgecolor=None)
    plt.plot([min(y_train), max(y_train)], [min(y_train), max(y_train)], color="red", linestyle="--", label="Perfect Prediction")
    plt.title("Train Set", fontsize=14)
    plt.xlabel("True Values", fontsize=12)
    plt.ylabel("Predicted Values", fontsize=12)
    plt.text(
        0.05, 0.95,  # Position dans le graphique (proportions)
        f"MSE: {mse_train:.2f}\nR²: {r2_train:.2f}",
        fontsize=10,
        ha="left",
        va="top",
        transform=plt.gca().transAxes,
        bbox=dict(facecolor='white', alpha=0.8, edgecolor='black')
    )
    plt.grid(True)
    plt.legend()

    # Graphique 2 : Ensemble de test
    plt.subplot(1, 2, 2)
    sns.scatterplot(x=y_test, y=y_pred_test, alpha=0.6, edgecolor=None)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color="red", linestyle="--", label="Perfect Prediction")
    plt.title("Test Set", fontsize=14)
    plt.xlabel("True Values", fontsize=12)
    plt.ylabel("Predicted Values", fontsize=12)
    plt.text(
        0.05, 0.95,  # Position dans le graphique (proportions)
        f"MSE: {mse_test:.2f}\nR²: {r2_test:.2f}",
        fontsize=10,
        ha="left",
        va="top",
        transform=plt.gca().transAxes,
        bbox=dict(facecolor='white', alpha=0.8, edgecolor='black')
    )
    plt.grid(True)
    plt.legend()

    # Sauvegarde de la figure
    plt.tight_layout()
    plt.savefig(save_path, dpi=300)
    plt.close()  # Fermer la figure après sauvegarde

In [ ]:
plot_train_test_predictions(
    y_train=y_train,
    y_test=y_test,
    X_train=X_train,
    X_test=X_test,
    model=xgb_model,
    save_path="train_test_scatter_plots_with_metrics.png"
)

In [ ]:
import joblib
def save_model(pipeline, filename):
    """Sauvegarde le modèle dans un fichier"""
    joblib.dump(pipeline, filename)
    print(f"Modèle sauvegardé sous {filename}")

def load_model(filename):
    """Charge un modèle depuis un fichier"""
    return joblib.load(filename)

save_model(xgb_model, 'xgboost_model.pkl')  # Sauvegarde

# Pour charger et utiliser le modèle sauvegardé
loaded_model = load_model('xgboost_model.pkl')
y_pred_2 = loaded_model.predict(X_test)

In [ ]:
print("Mean Squared Error (MSE):", mean_squared_error(y_pred_2, y_pred))
print("R2 Score:", r2_score(y_pred_2, y_pred))
sns.scatterplot(x=y_pred_2,y=y_pred)

In [14]:
from sklearn.model_selection import GridSearchCV

def grid_search_xgboost(pipeline, X_train, y_train):
    # Définition des paramètres à tester dans la grille
    param_grid = {
        'model__n_estimators': [100, 300, 500],         # Nombre d'arbres
        'model__learning_rate': [0.01, 0.05, 0.1],     # Taux d'apprentissage
        'model__max_depth': [5, 10, 15],               # Profondeur maximale
        # 'model__subsample': [0.8, 1.0],                # Sous-échantillonnage des données
        # 'model__colsample_bytree': [0.8, 1.0],         # Sous-échantillonnage des colonnes
        'model__gamma': [0, 1, 5],                     # Régularisation
    }

    # Configuration de la GridSearchCV
    grid_search = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        cv=5,                  # Nombre de folds pour la validation croisée
        scoring='neg_mean_squared_error',  # Métrique d'évaluation
        n_jobs=2,
        verbose=1              # Affichage des étapes
    )

    # Exécution de la recherche
    grid_search.fit(X_train, y_train)

    return grid_search

pipeline = build_xgboost_model()


In [15]:
# xgbest_model = grid_search_xgboost(pipeline, X_train, y_train)
# y_pred = xgbest_model.predict(X_test)
# print("Mean Squared Error (MSE):", mean_squared_error(y_test, y_pred))
# print("R2 Score:", r2_score(y_test, y_pred))
# sns.scatterplot(x=y_test,y=y_pred)

In [7]:
import polars as pl
df_final = pl.read_csv("data_processed/data_dvf_preprocessed.csv",columns=[
                                        'surface_reelle_bati', 'year','type_local', 'sin_month', 'cos_month', 'nombre_lots',
                                        'total_surface_carrez', 'lot1_surface_carrez', 'lot2_surface_carrez',
                                        'lot3_surface_carrez', 'lot4_surface_carrez', 'lot5_surface_carrez',
                                        'nombre_pieces_principales', 'surface_terrain', 'longitude', 'latitude','valeur_fonciere',
                                        'densite_weighted', 'transport_pois_weighted', 'education_pois_weighted',
                                        'health_pois_weighted', 'food_pois_weighted', 'shopping_pois_weighted',
                                        'park_pois_weighted', 'entertainment_pois_weighted', 'cultural_pois_weighted'
                                        ],
                                    schema_overrides={
                                            'surface_reelle_bati':pl.Float32,
                                            'type_local':pl.Utf8,
                                            'year':pl.Float32,
                                            'sin_month':pl.Float32,
                                            'cos_month':pl.Float32,
                                            'nombre_lots':pl.Float32,
                                            'total_surface_carrez':pl.Float32,
                                            'lot1_surface_carrez':pl.Float32,
                                            'lot2_surface_carrez':pl.Float32,
                                            'lot3_surface_carrez':pl.Float32,
                                            'lot4_surface_carrez':pl.Float32,
                                            'lot5_surface_carrez':pl.Float32,
                                            'nombre_pieces_principales':pl.Float32,
                                            'surface_terrain':pl.Float32,
                                            'longitude':pl.Float32,
                                            'latitude':pl.Float32,
                                            'valeur_fonciere':pl.Float32,
                                            'densite_weighted':pl.Float32,
                                            'transport_pois_weighted':pl.Float32,
                                            'education_pois_weighted':pl.Float32,
                                            'health_pois_weighted':pl.Float32,
                                            'food_pois_weighted':pl.Float32,
                                            'shopping_pois_weighted':pl.Float32,
                                            'park_pois_weighted':pl.Float32,
                                            'entertainment_pois_weighted':pl.Float32,
                                            'cultural_pois_weighted':pl.Float32
                                        },
                                    ignore_errors=True,)

In [ ]:
df_final.null_count()

In [ ]:
import polars as pl
import numpy as np

# Création d'un DataFrame Polars avec NaN dans col_1
df = pl.DataFrame({
    "col_1": [np.nan],
    "col_2": [3]
})

# Remplacer NaN dans 'col_1' par 0 avant l'addition
df = df.with_columns(
    (pl.col("col_1") + pl.col("col_2") > 2).alias("result")
)

print(df)
